In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(2)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/Unet_RC5')

dpath = DATA_PATH/(proj_id + '_82222_D.h5')
gpath = DATA_PATH/(proj_id + '_82222_G.h5')
keep_pct=1.0
bs=24
sz=256
wd=1e-7
lr=5e-3
lrs = np.array([lr/1000,lr/100,lr])

torch.backends.cudnn.benchmark=True

## Training

In [4]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [5]:
netG = Unet34(nf_factor=2).cuda()
netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
unet = LearnerGenModuleWrapper(netG, 'unet')
#load_model(netG, gpath)
learn = ConvLearner(md, unet)
imgGenVis = ImageGenVisualizationCallback(TENSORBOARD_PATH, netG, md, 'learner')
learn.metrics = []
learn.opt_fn=optim.Adam
#learn.crit = F.mse_loss #(turns sepia/blurry)
learn.crit = FeatureLoss()

In [6]:
#learn.freeze_to(1)

In [7]:
#learn.lr_find(1e-4, 1e1, wds=wd, linear=False)

In [8]:
#learn.sched.plot(n_skip=0, n_skip_end=0)

In [ ]:
learn.fit(lrs, 1, cycle_len=1, wds=wd, use_clr_beta=(20,10,0.95,0.85), callbacks=[imgGenVis])

/media/jason/Projects/Deep Learning/Deep Learning Projects/Jupyter Experiments/1_BlackWhiteToColor/fasterai/generators.py:131: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h<target_h or w<target_w:


Failed to generate graph for model: %426 : Long() = onnx::Constant[value={2}]()
 has empty scope name. Note that there's an outstanding issue with scopes being addressed here:  https://github.com/pytorch/pytorch/pull/12400


 15%|█▍        | 7897/52849 [1:22:33<7:49:58,  1.59it/s, loss=0.0498]

In [ ]:
vis = ModelImageVisualizer()

In [ ]:
vis.plot_transformed_image("data/uploaded_images/einstein_beach.jpg", netG)